In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os

hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [9]:
links = ["https://relevan.com.my/category/kesihatan/",
        "https://relevan.com.my/category/kewangan/",
        "https://relevan.com.my/category/pendidikan/",
        "https://relevan.com.my/category/kerjaya/"]

In [14]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            if r.status_code == 508:
                print(r.status_code)
                print('error in link, retrying... ', url)
                time.sleep(20)
                continue
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")
    
    main = soup.find("div",attrs = {'class':'col-8 main-content'})
    
    if main is None:
        return

    block = main.find_all("h2",attrs = {"class":"is-title post-title"})
    
    if block is None:
        return

    for link in main.find_all('h2',attrs = {"class":"is-title post-title"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [15]:
max_worker = 10
hrefs = []

for url in links:
    
    print(url)
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page')[1].replace('/','')) for a_ in pgno])
    print(max_page)
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}/page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


https://relevan.com.my/category/kesihatan/
64


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.29s/it]


https://relevan.com.my/category/kewangan/
17


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.96s/it]


https://relevan.com.my/category/pendidikan/
30


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.60s/it]


https://relevan.com.my/category/kerjaya/
16


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  2.00s/it]


In [18]:
len(hrefs)

1245

In [17]:
hrefs = list(set(hrefs))

In [19]:
with open('relevan.com.my-links.json', 'a') as f:
    json.dump(hrefs, f)

In [6]:
with open('relevan.com.my-links.json') as f:
    hrefs = json.load(f)

In [28]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
    comments = []
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="is-title post-title").text
        h = soup.find('div', attrs = {"class":"post-content-wrap"})
        p = [p.text.replace('\xa0', '') for p in h.find_all("p") if len(p.text.split(" ")) > 3]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None
    
    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [30]:
max_workers = 10

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('relevan.com-pages.jsonl', 'a',encoding='utf8') as f:
                json.dump(result, f,ensure_ascii=False)
                f.write('\n')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1245/1245 [02:27<00:00,  8.45it/s]


In [29]:
process_url("https://relevan.com.my/2023/06/05/pemindahan-sel-stem-mampu-panjangkan-usia/")

{'url': 'https://relevan.com.my/2023/06/05/pemindahan-sel-stem-mampu-panjangkan-usia/',
 'headline': 'Pemindahan sel stem mampu panjangkan usia',
 'content': ['SEBUAH pengubahsuaian genetik dibuktikan mampu melindungi tikus daripada menghidap kanser. Pengubahsuaian itu juga membolehkan haiwan itu hidup sehingga 20 tahun lebih lama, kekal sihat sehingga mencapai usia lebih matang.',
  'Kelebihan untuk meningkatkan jangka hayat tikus itu juga boleh dipindahkan kepada tikus-tikus lain melalui pemindahan sel stem darah.',
  'Seorang pakar di Universiti Perubatan Taipei yang dikenali sebagai Shen memberitahu, dia dan pasukannya setakat ini masih belum menemui sebarang kesan sampingan negatif daripada penemuan ini.',
  'Shen yang mengetuai penyelidikan itu telah bekerjasama dengan firma bioteknologi untuk menambah baik rawatan kanser pada manusia.',
  'Sebelum ini, pemindahan sel stem darah sudah menjadi sebahagian daripada rawatan kanser darah tertentu. Mereka berharap, kaedah pengubahsuaia